In [201]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from GoogleNews import GoogleNews
from newspaper import Article
from newspaper import Config
from wordcloud import WordCloud, STOPWORDS
import snscrape.modules.twitter as sntwitter
import re
from flask import Flask,request, url_for, redirect, render_template, jsonify,flash
import pickle
import numpy as np

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pande\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [215]:
pip list

Package                            Version
---------------------------------- -------------------
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.9.1
anyio                              2.2.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.5
astropy                            4.2.1
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.6
Babel                              2.9.0
backcall                           0.2.0
backports.functools-lru-cache      1.6.4
backports.shutil-get-terminal-size 1.0.0
backports.tempfile                 1.0
backports.weakref                  1.0.post1
backports.zoneinfo                 0.2

In [202]:
def config():
    nltk.download('punkt')
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'
    config = Config()
    config.browser_user_agent = user_agent
    config.request_timeout = 10
    
    return config

In [203]:
def newsGoogle(CompanyName,yesterday,now):
    import time
    #As long as the company name is valid, not empty...
    if CompanyName != '':
        print(f'Searching for and analyzing {CompanyName}, Please be patient, it might take a while...')
        #Extract News with Google News
        googlenews = GoogleNews(start='01/11/2022',end='01/11/2022')
        #googlenews = GoogleNews(period='1d')
        googlenews.search(CompanyName)
        result = googlenews.result()
        df_New = pd.DataFrame()
        page_init=1
        while (page_init < 2):
            result=googlenews.page_at(page_init)
            page_init+=1
            time.sleep(3)
            #store the results
            df_New = df_New.append(result)
    return df_New

In [204]:
def newsPerprocess(companyName):
    now = dt.date.today()
    now = now.strftime('%m-%d-%Y')
    yesterday = dt.date.today() - dt.timedelta(days = 2)
    print(yesterday)
    yesterday = yesterday.strftime('%m-%d-%Y')
    dfTest=newsGoogle(companyName,yesterday,yesterday)
    print(dfTest)
    try:
        from datetime import datetime
        dfTest['DateNew']=dfTest['datetime'].dt.date
        end = datetime.now()
        end=end.date()
        print(end)
        dfTest['DateNew'] = dfTest['DateNew'].fillna(end)
        list =[] #creating an empty list 
        for i in dfTest.index:
            dict = {} #creating an empty dictionary to append an article in every single iteration
            config=Config()
            article = Article(dfTest['link'][i],config=config) #providing the link
            try:
                article.download() #downloading the article 
                article.parse() #parsing the article
                article.nlp() #performing natural language processing (nlp)
            except:
                pass 
        #storing results in our empty dictionary
            dict['Date']=dfTest['DateNew'][i]
            dict['Media']=dfTest['media'][i]
            dict['Title']=article.title
            dict['Article']=article.text
            dict['Summary']=article.summary
            dict['Key_words']=article.keywords
            list.append(dict)
        check_empty = not any(list)
    # print(check_empty)
        if check_empty == False:
            news_df=pd.DataFrame(list) #creating dataframe
            print(news_df)

    except Exception as e:
    #exception handling
        #print("exception occurred:" + str(e))
        print('Looks like, there is some error in retrieving the data, Please try again or try with a different ticker.' )
        return
    newArt=news_df.groupby(['Date'], as_index = False).agg({'Summary': ','.join})
    return newArt

In [205]:
def cleanTxt(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    #text = re.sub(' ', '', text)
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    return text

In [208]:
def getTweets(query,noOfTweets):
    query = query
    #As long as the query is valid (not empty or equal to '#')...
    if query != '':
        noOfTweet = noOfTweets
        if noOfTweet != '' :
            noOfDays = 1
            if noOfDays != '':
                #Creating list to append tweet data
                tweets_list = []
                now = dt.date.today()
                now = now.strftime('%Y-%m-%d')
                yesterday = dt.date.today() - dt.timedelta(days = int(noOfDays))
                yesterday = yesterday.strftime('%Y-%m-%d')
                for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query + ' lang:en since:' +  yesterday + ' until:' + now + ' -filter:links -filter:replies').get_items()):
                    if i > int(noOfTweet):
                        break
                    tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.username])

                #Creating a dataframe from the tweets list above 
                df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
                print(df)
                df["Text"] = df["Text"].apply(cleanTxt)
                df['DateNew'] = pd.to_datetime(df['Datetime']).dt.date
                df_new=df.groupby(['DateNew'], as_index = False).agg({'Text': ','.join})
                df_new.columns=['Date','Summary']
                return df_new

In [209]:
def combineData(tweetText,ArtText):
    NewCombineDF=pd.concat([tweetText,ArtText])
    NewCombineDF['Date']=pd.to_datetime(NewCombineDF['Date'])
    NewCombineDF=NewCombineDF.groupby(['Date'], as_index = False).agg({'Summary': ','.join})
    NewCombineDF['Summary']=NewCombineDF['Summary'].str.cat(sep=' ')
    return NewCombineDF

In [210]:
def polarity(NewCombineDF):
    #Assigning Initial Values
    positive = 0
    negative = 0
    neutral = 0
#Creating empty lists
#news_list = []
    polmap={}

#Iterating over the tweets in the dataframe
    for news in NewCombineDF['Summary']:
    #news_list.append(news)
        analyzer = SentimentIntensityAnalyzer().polarity_scores(news)
        neg = analyzer['neg']
        neu = analyzer['neu']
        pos = analyzer['pos']
        comp = analyzer['compound']
    #print(neg,pos,neu,comp)
        polmap.setdefault('Negative',[]).append(neg)
        polmap.setdefault('Positive',[]).append(pos)
        polmap.setdefault('Neutral',[]).append(neu)
        polmap.setdefault('Comp',[]).append(comp)
    polDF=pd.DataFrame(polmap)
    return polDF

In [211]:
def finalData(NewCombineDF,PolDF):
    Pol=pd.concat([NewCombineDF,PolDF],axis=1)
    return Pol

In [212]:
def model(Article,Tweet):
    ArtText=pd.DataFrame()
    ArtText=newsPerprocess(Article)
    tweetText=getTweets(Tweet,1000)
    NewCombineDF=combineData(tweetText,ArtText)
    poldf=polarity(NewCombineDF)
    final=finalData(NewCombineDF,poldf)
    return final

In [213]:
# Initalise the Flask app
import pickle
app = Flask(__name__)
poldf=pd.DataFrame()

In [120]:
@app.route('/')
def home():
    return render_template('stock.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    article = request.form.get("Articles")
    tweets = request.form.get("Tweets")
    poldf=model(article,tweets)
    # load saved model
    with open('model_arima_pkl' , 'rb') as f:
        m_ar = pickle.load(f)
    # load saved model
    with open('rfc_pkl' , 'rb') as f:
        rfc_mod = pickle.load(f)
    
    print(poldf)
    positive=poldf[['Positive','Negative']]
    
    predicted_val=0.60*rfc_mod.predict(positive)+(1-0.60)*m_ar.predict(1)
    predicted_val
    return render_template('stock.html', prediction_text='Your Predicted Stock Price is: {}'.format(predicted_val))

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jan/2022 23:02:05] "GET / HTTP/1.1" 200 -


2022-01-10
Searching for and analyzing Reliance Industries, Please be patient, it might take a while...
                                               title                    media  \
0  France Plans $1.1 Billion to Safeguard Metals ...                            
1  LIC IPO could make the PSU India's second larg...       The Economic Times   
2  How Amazon's retail battle with Reliance turne...    Retail News from Asia   
3  Amazon’s battle with addiction for retail supr...  The Bharat Express News   
4  Equities extend gains, Q3FY22 earnings in focu...        The Assam Tribune   
5  Business News | Sensex Surges 504 Points, Soar...                 LatestLY   
6  Sensex Rises Over 500 Points, Nifty Reclaims 1...            Outlook India   
7  Amazon vs Reliance In Retail - The Big And Exp...         BW Businessworld   
8  Sensex Rises Over 300 Points, Nifty Above 18,1...            Outlook India   
9   Reliance Industries Trades Higher On The Bourses             Udaipur News   

    

127.0.0.1 - - [12/Jan/2022 23:02:30] "POST /predict HTTP/1.1" 200 -


Empty DataFrame
Columns: [Datetime, Tweet Id, Text, Username]
Index: []
        Date                                            Summary  Negative  \
0 2022-01-12  Procuring enough raw materials for EV batterie...     0.024   

   Positive  Neutral    Comp  
0     0.115    0.861  0.9969  


In [214]:
@app.route('/')
def index():
    return render_template('base.html')

@app.route('/login', methods=['GET','POST'])
def login():
    error = None
    if request.method == 'POST':
        if request.form['username'] != 'admin' or \
                request.form['password'] != 'secret':
            error = 'Invalid credentials'
        else:
            #flash('You were successfully logged in')
            return redirect(url_for('predict'))
    return render_template('login.html', error=error)

@app.route('/pred',methods=['POST','GET'])
def predict():    
    if request.method == 'POST':
        article = request.form.get("Articles")
        print(article)
        tweets = request.form.get("Tweets")
        print(tweets)
        poldf=model(article,tweets)
        # load saved model
        with open('model_arima_pkl' , 'rb') as f:
            m_ar = pickle.load(f)
        # load saved model
        with open('rfc_pkl' , 'rb') as f:
            rfc_mod = pickle.load(f)
        print(poldf)
        positive=poldf[['Positive','Negative']]
        predicted_val=0.60*rfc_mod.predict(positive)+(1-0.60)*m_ar.predict(1)
        predicted_val
        return render_template('predict1.html', prediction_text='Your Predicted Stock Price is: {}'.format(predicted_val))
    return render_template('predict1.html')
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Jan/2022 02:27:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jan/2022 02:27:19] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [13/Jan/2022 02:27:21] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [13/Jan/2022 02:27:21] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [13/Jan/2022 02:27:25] "GET /pred HTTP/1.1" 200 -
127.0.0.1 - - [13/Jan/2022 02:27:25] "GET /static/style.css HTTP/1.1" 404 -


Reliance Industries
Reliance Stock
2022-01-11
Searching for and analyzing Reliance Industries, Please be patient, it might take a while...
                                               title                    media  \
0  France Plans $1.1 Billion to Safeguard Metals ...                            
1  LIC IPO could make the PSU India's second larg...       The Economic Times   
2  How Amazon's retail battle with Reliance turne...    Retail News from Asia   
3  Amazon’s battle with addiction for retail supr...  The Bharat Express News   
4  Equities extend gains, Q3FY22 earnings in focu...        The Assam Tribune   
5  Business News | Sensex Surges 504 Points, Soar...                 LatestLY   
6  Sensex Rises Over 500 Points, Nifty Reclaims 1...            Outlook India   
7  Amazon vs Reliance In Retail - The Big And Exp...         BW Businessworld   
8  Sensex Rises Over 300 Points, Nifty Above 18,1...            Outlook India   
9   Reliance Industries Trades Higher On The Bourse

<ipython-input-208-dea745a4fcb7>:18: FutureWarning: username is deprecated, use user.username instead
  tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.username])
127.0.0.1 - - [13/Jan/2022 02:27:54] "POST /pred HTTP/1.1" 200 -
127.0.0.1 - - [13/Jan/2022 02:27:54] "GET /static/style.css HTTP/1.1" 404 -


                    Datetime             Tweet Id  \
0  2022-01-12 16:47:23+00:00  1481306826251460609   
1  2022-01-12 10:35:04+00:00  1481213129577365515   
2  2022-01-12 09:11:34+00:00  1481192117305286656   
3  2022-01-12 08:14:38+00:00  1481177788857995265   
4  2022-01-12 08:09:48+00:00  1481176572216360960   
5  2022-01-12 07:57:30+00:00  1481173478086955009   
6  2022-01-12 07:30:03+00:00  1481166569816608770   
7  2022-01-12 06:57:41+00:00  1481158425669423106   
8  2022-01-12 05:53:19+00:00  1481142224822083587   
9  2022-01-12 05:20:29+00:00  1481133964031295488   
10 2022-01-12 05:17:12+00:00  1481133137258479616   
11 2022-01-12 05:09:41+00:00  1481131245392175104   
12 2022-01-12 01:40:59+00:00  1481078724258930701   

                                                 Text         Username  
0   #Sintex\n\nLot of people asking whether to inv...        Harrshit1  
1   🐂Top 5 Gainers on 12-Jan-2022 in NIFTY50📈\n\nM...      SiriusStats  
2   Top Gainers\n\nM&amp;M          4.